<a href="https://colab.research.google.com/github/RomaricKanyamibwa/Normalisation-de-concepts-medicaux/blob/master/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install --upgrade gensim

In [0]:

import pandas as pd
#import collections
from pprint import pprint  # pretty-printer
import gensim
from gensim.models import word2vec
from gensim.parsing.preprocessing import remove_stopwords
import logging

In [0]:
%pwd
%cd home/
%ls -l

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


In [35]:

!cd drive/Coolab/
!ls -l

total 4
drwxr-xr-x 2 root root 4096 Nov 25 23:35 drive


In [0]:
df_disorders=pd.read_csv("drive/Coolab/Disorders.csv",names=["Code","Disorder","GenDisorder"],sep="\t") 
Codes=df_disorders['Code']#Y vector (outputs)
Disorders=df_disorders['Disorder']#X vector(test)
GenDisorders=df_disorders['GenDisorder']#X vector (train)

In [0]:
def input_tokenize(input_list):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_list))
    
    for i, line in enumerate (input_list): 
        if (i%10000==0):
            logging.info ("read {0} reviews".format (i))
        # do some pre-processing and return a list of words for each review text
        line=remove_stopwords(line)
        yield gensim.utils.simple_preprocess(line,min_len=3)

for i,k in enumerate(input_tokenize(Disorders)):
  print(k)
  if i>10:break

In [0]:
logging.info ("Tokenization")
Disorders_corpus=(input_tokenize(Disorders))
GenDisorders_corpus=(input_tokenize(GenDisorders))
logging.info ("Done Tokenization")

Corpus=[[m]+n for m,n in zip(Codes,GenDisorders_corpus)]


In [55]:

for i,k in enumerate(Corpus):
  print(k)
  if i>10:break

print("Iterator")
iter_Corpus=iter(Corpus)
for i,k in enumerate(iter_Corpus):
  print(k)
  if i>10:break

['C0423346', 'cupuliform', 'cataract']
['C4510595', 'acquired', 'hallux', 'limitus', 'right', 'great', 'toe']
['C1394389', 'cyst', 'ora', 'serrata', 'retina']
['C1394389', 'cyst', 'ora', 'serrata', 'retina']
['C1394389', 'cyst', 'ora', 'serrata', 'retina']
['C1394389', 'cyst', 'ora', 'serrata', 'retina']
['C1394389', 'cyst', 'ora', 'serrata', 'retina']
['C1394389', 'cyst', 'ora', 'serrata', 'retina']
['C0520729', 'subconjunctival', 'cyst']
['C0332603', 'leonine', 'facies']
['C0333292', 'perforated', 'ulcer']
['C0333292', 'perforated', 'ulcer']
Iterator
['C0423346', 'cupuliform', 'cataract']
['C4510595', 'acquired', 'hallux', 'limitus', 'right', 'great', 'toe']
['C1394389', 'cyst', 'ora', 'serrata', 'retina']
['C1394389', 'cyst', 'ora', 'serrata', 'retina']
['C1394389', 'cyst', 'ora', 'serrata', 'retina']
['C1394389', 'cyst', 'ora', 'serrata', 'retina']
['C1394389', 'cyst', 'ora', 'serrata', 'retina']
['C1394389', 'cyst', 'ora', 'serrata', 'retina']
['C0520729', 'subconjunctival', 'cyst

In [0]:
def gensim_demo(sentences):
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    model = word2vec.Word2Vec(sentences, iter=10, min_count=1, size=500, workers=6)

    # get the word vector of "the"
    #print(model.wv['the'])

    # get the most common words
    print(model.wv.index2word[0], model.wv.index2word[1], model.wv.index2word[2])

    # get the least common words
    vocab_size = len(model.wv.vocab)
    print(model.wv.index2word[vocab_size - 1], model.wv.index2word[vocab_size - 2], model.wv.index2word[vocab_size - 3])

    # find the index of the 2nd most common word ("of")
#    print('Index of "of" is: {}'.format(model.wv.vocab['of'].index))

    # some similarity fun
    print(model.wv.similarity('fracture', 'bone'), model.wv.similarity('broken', 'arm'))

    # what doesn't fit?
    print(model.wv.doesnt_match("green blue red zebra".split()))

#    str_data = read_data(root_path + filename)
#    index_data = convert_data_to_index(str_data, model.wv)
#    print(str_data[:4], index_data[:4])

    # save and reload the model
#    model.save("Disorders")
    return model



In [63]:

#print("size corpus:",len(list(iter_Corpus)))
logging.info ("Tokenization")
Disorders_corpus=(input_tokenize(Disorders))
GenDisorders_corpus=(input_tokenize(GenDisorders))
logging.info ("Done Tokenization")
#pense à cree des fichier avec les donées


Corpus=[[m]+n for m,n in zip(Codes,GenDisorders_corpus)]
iter_Corpus=(Corpus)

model=gensim_demo(iter_Corpus)
model.most_similar('fracture')

fracture encounter unspecified
C2214472 ivc C0857024
0.48722538 0.060895473


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


zebra


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.


[('routine', 0.5865849852561951),
 ('nonunion', 0.5744647979736328),
 ('closed', 0.5593452453613281),
 ('delayed', 0.5548031330108643),
 ('dislocation', 0.5514476299285889),
 ('malunion', 0.5488807559013367),
 ('healing', 0.5481484532356262),
 ('bone', 0.48722541332244873),
 ('displacement', 0.48619863390922546),
 ('nondisplaced', 0.476291686296463)]